In [6]:
import numpy as np
import pandas as pd
from numpy.random import random as rand
from tqdm import tqdm 
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [16]:
ss = StandardScaler()
data = np.loadtxt(r'dataset/australian.dat')
b = np.ones(data.shape[0])
ones=np.ones(data.shape[0])
data = np.column_stack((ones,data))
for i in range(data.shape[0]):
    if data[:,-1][i]==0:
        data[:,-1][i]=-1
X_test=data[-600:,:-1]
y_test=data[-600:, -1]
X=data[:-600,:-1]
y=data[:-600:, -1]
X=ss.fit_transform(X)
X_test=ss.fit_transform(X_test)

In [12]:
import numpy as np

'''
蝙蝠算法-Bat Algorithm
'''


class BA(object):
    def __init__(self,d,hidden,N_p, N_gen, Qmin, Qmax, lower_bound, upper_bound, func):
        self.d = d*hidden  # 搜索维度
        self.dog=d
        self.hidden=hidden
        self.N_p = N_p  # 个体数
        self.N_gen = N_gen  # 迭代次数
        self.A = 1 + np.random.random(self.N_p)  # 响度
        self.r = np.random.random(self.N_p)  # 脉冲发射率
        self.Qmin = Qmin  # 最小频率
        self.Qmax = Qmax  # 最大频率
        self.lower_bound = lower_bound  # 搜索区间下限
        self.upper_bound = upper_bound  # 搜索区间上限
        self.func = func  # 目标函数
        self.alpha = 0.85
        self.gamma = 0.9
        self.r0 = self.r

        self.Lb = self.lower_bound * np.ones(self.d)
        self.Ub = self.upper_bound * np.ones(self.d)
        self.Q = np.zeros(self.N_p)  # 频率
        self.v = np.zeros((self.N_p, self.d))  # 速度

        self.sol = np.zeros((self.N_p, self.d))  # 种群
        self.fitness = np.zeros(self.N_p)  # 个体适应度
        self.best = np.zeros(self.d)  # 最好的solution

        self.fmin = 0.0  # 最小fitness

    # 初始化蝙蝠种群
    def init_bat(self):
        for i1 in tqdm(range(self.N_p)):
            self.sol[i1, :] = self.Lb + (self.Ub - self.Lb) * np.random.uniform(0, 1, self.d)
            self.fitness[i1] = self.func(self.sol[i1, :],d=self.dog,hidden=self.hidden)[0]
        self.fmin = np.min(self.fitness)
        fmin_arg = np.argmin(self.fitness)
        self.best = self.sol[fmin_arg, :]

    # 越界检查
    def simplebounds(self, s, lb, ub):
        for j1 in range(self.d):
            if s[j1] < lb[j1]:
                s[j1] = lb[j1]
            if s[j1] > ub[j1]:
                s[j1] = ub[j1]
        return s

    # 迭代部分
    def start_iter(self):
        S = np.zeros((self.N_p, self.d))
        self.init_bat()
        for step in tqdm(range(self.N_gen)):
            for i2 in range(self.N_p):
                self.Q[i2] = self.Qmin + (self.Qmin - self.Qmax) * np.random.uniform(0, 1)
                self.v[i2, :] = self.v[i2, :] + (self.sol[i2, :] - self.best) * self.Q[i2]
                S[i2, :] = self.sol[i2, :] + self.v[i2, :]
                S[i2, :] = self.simplebounds(S[i2, :], self.Lb, self.Ub)  # 越界检查

                if np.random.random() > self.r[i2]:
                    S[i2, :] = self.best + 0.001 * np.random.randn(self.d)  # 此处没有实现乘以响度平均值
                    S[i2, :] = self.simplebounds(S[i2, :], self.Lb, self.Ub)  # 越界检查

                Fnew = self.func(S[i2, :],d=self.dog,hidden=self.hidden)
                if (Fnew[0] <= self.fitness[i2]) and (np.random.random() < self.A[i2]):
                    self.sol[i2, :] = S[i2, :] 
                    self.fitness[i2] = Fnew[0]
                    self.A[i2] = self.alpha * self.A[i2]  # 响度更新
                    self.r[i2] = self.r0[i2] * (1 - np.exp(-1 * self.gamma * step))  # 脉冲发射率更新

                if Fnew[0] <= self.fmin:
                    self.best = S[i2, :]
                    self.fmin = Fnew[0]
                    self.b=Fnew[1]
#                     print('Fnew{}替代了self_fmin{}'.format(Fnew,self.fmin))
#             print(step, ':', '\n', 'BEST=', self.best)
#             print('\n', 'min of fitness=', self.fmin)
        
            asd=np.asarray(X_test)      
            predict=predict2_testBABP(self.best,self.dog,self.hidden,self.b)
            end=predict
            sum=0
            for i in range(predict.size):
                if predict[i]>0:
                    predict[i]=1
                else:
                    predict[i]=-1
            for i in range(predict.size):
                if predict[i]==y_test[i]:
                    sum+=1
            rate=sum/len(predict)
            print(rate)
#             print(step, ':', '\n', 'BEST=', self.best, '\n', 'min of fitness=', self.fmin)
        return self.best,self.fmin,self.b,end

In [9]:
def fp(x,p=5):
    exp1=4*p-2;exp2=1/(4*p-2)
    return x/(0.000001+(1+x**(exp1))**exp2)

#定义第一个损失函数，没有正则化
def function1_testBABP(W,d,hidden):#含有权重
    W=W.reshape(d,hidden)
    H_in=X.dot(W)
    H_out=fp(H_in)    
    if np.linalg.det(H_in.T.dot(H_in))==0:
        H_plus=np.linalg.inv((H_in.T.dot(H_in))).dot(H_in.T)
    else:
        HX=H_in.T.dot(np.linalg.inv((H_in.dot(H_in.T))))
    b=HX.dot(y)

    sum2=0
    for j in range(X.shape[0]):
        sum1=0
        for i in range(W.shape[1]):#默认权重从w0常数项开始，包括了偏执b
            sum1+=b[i]*fp(W[:,i].dot(X[j]))
        error=sum1-y[j]
        sum2+=(error)**2
    F=np.sqrt(sum2/X.shape[0])
    return F

#定义预测函数，无正则化
def predict1_testBABP(W,d,hidden,C=0.1):#含有权重
    W=W.reshape(d,hidden)
    H_in=X.dot(W)
    H_out=fp(H_in)    
    if np.linalg.det(H_in.T.dot(H_in))==0:
        H_plus=np.linalg.inv((H_in.T.dot(H_in))).dot(H_in.T)
    else:
        HX=H_in.T.dot(np.linalg.inv((H_in.dot(H_in.T))))
    b=HX.dot(y_test)    
    fx=H_in.dot(b)
    return fx


#定义第二个损失函数，进行正则化，大样本使用的损失函数
def function2_testBABP(W,d,hidden,C=0.1):#含有权重
    W=W.reshape(d,hidden)
    H_in=X.dot(W)
    H_out=fp(H_in)    
    HH= H_out.T.dot(H_out); HT = H_out.T.dot(y)
    b=np.linalg.pinv(HH+np.identity(W.shape[1])/C).dot(HT)
    sum2=0
    for j in range(X.shape[0]):
        sum1=0
        for i in range(W.shape[1]):#默认权重从w0常数项开始，包括了偏执b
            sum1+=b[i]*fp(W[:,i].dot(X[j]))
        error=sum1-y[j]
        sum2+=(error)**2
    F=np.sqrt(sum2/X.shape[0])
    return (F,b)
#定义预测函数，含有正则化
def predict2_testBABP(W,d,hidden,b,C=0.1):#含有权重
    W=W.reshape(d,hidden)
    H_in=X_test.dot(W)
    fx=H_in.dot(b)
    return fx

#定义LR损失函数
def function1_testBA1(W):
    sum2=0
    for i in range(X.shape[0]):
        sum1=0
        for j in range(W.size):
            sum1=sum1+X[i][j]*W[j]
        error=sum1-y[i]
        sqr_error=error**2
        sum2=sum2+sqr_error
    output=1/2*sum2
    mse=((error)**2)/X.shape[0]
    return output

In [17]:
ba = BA(d=15,hidden=88,N_p=50,N_gen=1000,Qmin=0,Qmax=2,lower_bound=-2,upper_bound=2,func=function2_testBABP)
best,fmin,b,end= ba.start_iter()

  0%|          | 1/1000 [00:02<47:28,  2.85s/it]

0.85


  0%|          | 2/1000 [00:05<46:32,  2.80s/it]

0.8483333333333334


  0%|          | 3/1000 [00:08<46:02,  2.77s/it]

...

0.8333333333333334


100%|█████████▉| 999/1000 [33:50<00:02,  2.11s/it]

0.8316666666666667


100%|██████████| 1000/1000 [33:52<00:00,  2.03s/it]

0.8333333333333334
